In [ ]:
# import localcider
# from localcider.sequenceParameters import SequenceParameters
# import protfasta
# import glob
import pandas as pd
import numpy as np
# import re
# import scipy
from scipy import stats
import matplotlib.pyplot as plt
# from collections import defaultdict 
# from Bio.Seq import Seq
# import Bio
# import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from scipy.signal import find_peaks

import seaborn as sns
lightblue , darkblue, lightgreen, darkgreen, pink =  '#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99'
# import editdistance

# import YeastAnalysisfunctions as YAF

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams['pdf.fonttype'] = 42

# alignment distance vs activity difference 
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import time

In [ ]:
masterDF = pd.read_csv('SC_Glucose_Ratio_MasterDF_20240930_withSeqFeatures.csv')
maxActivity = max(masterDF.Activity_SCglucose)
minActivity = min(masterDF[masterDF.Recovered].Activity_SCglucose)

Activity = 'Activity_SCglucose'
print('%i of %i Designed ADs recovered, %.1f percent'%(sum(masterDF['Recovered']),len(masterDF),100.0*sum(masterDF['Recovered'])/len(masterDF)))


# ActivityThresh = 100000
# ActivityThresh=45373 #new thresholds from Aaron's model June 2024
# HighActivityThresh=45373*2
ActivityThresh=  94031 # 80th percentile 20241001


#define WT
ScerWTactivity = np.mean(masterDF[masterDF.ADseq=='STDSTPMFEYENLEDNSKEWTSLFDNDIPVTTDDVSLADK'][Activity])
ScerWT_DF = masterDF[masterDF.ADseq=='STDSTPMFEYENLEDNSKEWTSLFDNDIPVTTDDVSLADK']

Orthologs_DF = pd.read_csv('../data/OrthologTilingDFwActivities_20240930.csv', index_col=0)

FullOrthoDF =  pd.read_pickle('../data/FullOrthologDF_20240930')

In [ ]:
Orthologs_DF

In [ ]:
def count_peak_ADs(tempDF):
    peaks, _ = find_peaks(tempDF["SmoothedActivitesLoess"], prominence=25000, height=ActivityThresh)
    if tempDF["SmoothedActivitesLoess"][0] > ActivityThresh + 25000:
        peaks = np.append(peaks, 0)

    return peaks, len(peaks)

# make a pretty WT Gcn4 plot

In [ ]:
# other features
Gcn4Features = {"uORF1":[-360./3,-349/3,'gray'] ,
'uORF2':[-292./3,-284/3,'gray'],
'uORF3':[-175./3,-164/3,'gray'],
'uORF4':[-150./3,-139/3,'gray'],
                'DBD':[225,279,'darkblue'],
                'N terminal AD':[0,100,'lightblue'],
                'Central acidic AD':[101,140,'orange']
               }
# 'CDS':[1..846]# (includes STOP)
tracecolor ='pink'
x = 'NativeLocation'# 
y = 'Activity'
halfwindow =20 
fig = plt.figure(figsize=[12,6])
ax = fig.add_subplot(1,1,1)
# Define the Scer WT for comparison
ScertempDF = pd.read_csv('Scer_WT_Data.csv')

ax.plot(ScertempDF[x],ScertempDF[y],c=tracecolor,linewidth=2,marker='.',label='AD activity')
# ax.scatter(ScertempDF[x],ScertempDF[y],c=tracecolor,linewidth=2,marker='o',label='AD activity')
ax.errorbar(ScertempDF[x],ScertempDF[y],yerr=ScertempDF['Stdev_BYS4_BioReps'],c='pink',mfc=tracecolor,linewidth=1,marker='.',mew=0,label='AD activity')


for i, entry in ScertempDF.iterrows():
    ax.plot([entry[x]+1-halfwindow,entry[x]+1+halfwindow],[entry[y],entry[y]],linewidth=2,c=tracecolor,alpha=.5)
ax.plot([-130,281],[ActivityThresh,ActivityThresh],c='gray')
tempDF = FullOrthoDF[FullOrthoDF.SpeciesName=='Saccharomycotina_sp|P03069|GCN4_YEAST']
# ax.plot(tempDF.SmoothedActivites.values[0],label='Average smoothening')
ax.plot(tempDF.SmoothedActivitesLoess.values[0],label='Loess Smoothening')
# ax.plot(tempDF.SmoothedActivites)

for i in Gcn4Features.keys():
    x1,x2,color = Gcn4Features[i]
    ax.plot([x1,x2],[1,1],c=color,linewidth=10,alpha=.5,label=i)
plt.xlabel('Position (AA)'),plt.ylabel('Activity')
plt.legend(loc=2)
# plt.savefig('../Results/AnnotatedGcn4_smoothing.pdf')
plt.show()

In [ ]:
domain_df = pd.read_table("../data/FullLengthUniqueGcn4Orthologs_domains.tsv", header=None)
columns = [
    "name",
    "sequence_md5_digest",
    "length",
    "analysis", # Pfam, PRINTS, Gene3D
    "signature_acession",
    "signature_description",
    "start",
    "end",
    "score",
    "status", # T if true?
    "date",
    "interpro_annot",
    "interpro_description",
    "GO_annotations",
    "Pathway_annotations"
]

domain_df.columns = columns
cerevisiae_gene = 'Saccharomycotina_sp|P03069|GCN4_YEAST'
cerevisiae_domain_df = domain_df[[f"{cerevisiae_gene}" in name for name in domain_df["name"]]]
SUPERFAMILY = cerevisiae_domain_df[cerevisiae_domain_df["analysis"] == "SUPERFAMILY"]["signature_description"].values
filtered_domain_df = domain_df[[(s in SUPERFAMILY) for s in domain_df["signature_description"]]]
domain_coordinates = filtered_domain_df[['name', 'start', 'end']]

In [ ]:
def get_overlapping_tile_ADs(tile_tempDF):
    ADs = []
    AD_end = 0
    AD_start = 0

    for i in tile_tempDF.index: 
        if tile_tempDF.loc[i, "Activity"] > ActivityThresh:
            # If start of new active tile is after the end of last active tile
            if tile_tempDF.loc[i, "NativeLocation"] - 19 > AD_end:
                # Save old AD and start a new AD
                if AD_end > 0: 
                    ADs.append((AD_start, AD_end))
                # Make new AD
                AD_end = tile_tempDF.loc[i, "NativeLocation"] + 20
                AD_start = tile_tempDF.loc[i, "NativeLocation"] - 19

            # Extend current AD
            else: 
                AD_end = tile_tempDF.loc[i, "NativeLocation"] + 20

    # Add the final AD to the list
    if AD_end > 0: 
        ADs.append((AD_start, AD_end))
    
    return ADs


In [ ]:
# Version for all species
def make_activity_trace(name, species, ax):
    tracecolor ='pink'
    x = 'NativeLocation'# 
    y = 'Activity'
    halfwindow =20 

    # Plot tiles first
    tile_tempDF = Orthologs_DF[Orthologs_DF["SpeciesNames"] == name].sort_values("NativeLocation").dropna(subset="Activity")

    ax.plot(tile_tempDF[x],tile_tempDF[y],c=tracecolor,linewidth=2,marker='.',label='AD activity')
    ax.errorbar(tile_tempDF[x],tile_tempDF[y],yerr=tile_tempDF['Stdev_BYS4_BioReps'],c='pink',mfc=tracecolor,linewidth=1,marker='.',mew=0)


    # Plotting tile lengths
    for i, entry in tile_tempDF.iterrows():
        ax.plot([entry[x]+1-halfwindow,entry[x]+1+halfwindow],[entry[y],entry[y]],linewidth=2,c=tracecolor,alpha=.5)

    tempDF = FullOrthoDF[FullOrthoDF.SpeciesName==name]
    
    # Plotting threshold
    ax.plot([0,len(tempDF.Seq.values[0])],[ActivityThresh,ActivityThresh],c='gray')

    # Plotting the blue line
    ax.plot(tempDF.SmoothedActivitesLoess.values[0],label='Loess Smoothening')

    # WxxLF location
    ax.plot([tempDF.WxxLF_loc,tempDF.WxxLF_loc+4],[1,1],c="orange",linewidth=10,alpha=.5)

    # DBD location
    DBD_tempDF = domain_coordinates[domain_coordinates["name"] == name]
    ax.plot([DBD_tempDF.start,DBD_tempDF.end],[1,1],c="darkblue",linewidth=10,alpha=.5)

    # AD locations
    AD_ls = get_overlapping_tile_ADs(tile_tempDF)
    for start, end in AD_ls:
        ax.plot([start,end],[-10000,-10000],c="lightblue",linewidth=10,alpha=.5)

    peaks, peak_len = count_peak_ADs(tempDF.iloc[0])

    # Add label with number of ADs
    if len(AD_ls) != 1:
        ax.text(
            0.98, 0.7, f'{len(AD_ls)} ADs (overlapping tiles)',
            transform=ax.transAxes,
            ha='right', va='top',  # horizontal and vertical alignment
            fontsize=12,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7)  # optional background
        )
    else: 
        ax.text(
            0.98, 0.7, f'{len(AD_ls)} AD (overlapping tiles)',
            transform=ax.transAxes,
            ha='right', va='top',  # horizontal and vertical alignment
            fontsize=12,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7)  # optional background
        )

    if peak_len != 1:
        ax.text(
            0.98, 0.6, f'{peak_len} AD peaks',
            transform=ax.transAxes,
            ha='right', va='top',  # horizontal and vertical alignment
            fontsize=12,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7)  # optional background
        )
    else: 
        ax.text(
            0.98, 0.6, f'{peak_len} AD peak',
            transform=ax.transAxes,
            ha='right', va='top',  # horizontal and vertical alignment
            fontsize=12,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7)  # optional background
        )


    plt.xlabel('Position (AA)'),plt.ylabel('Activity')

    # Manually adding AD to the legend
    
    handles, labels = ax.get_legend_handles_labels()
    custom_patch = Patch(color='lightblue', label='AD')
    handles.append(custom_patch)
    labels.append('AD')

    custom_patch = Patch(color='orange', label='WxxLF')
    handles.append(custom_patch)
    labels.append('WxxLF')

    custom_patch = Patch(color='darkblue', label='DBD')
    handles.append(custom_patch)
    labels.append('DBD')

    ax.legend(handles, labels, loc=1)
    ax.set_title(f"{species}: {name}")
    # plt.savefig(f'figures/{name}.pdf', format="pdf", bbox_inches='tight')
    # plt.close()

In [ ]:
name_key = pd.read_csv("../data/phylogenetic_info/species_mapping.csv")
name_key = name_key.rename({'SpeciesName' : 'Species'}, axis=1)
name_key = name_key.merge(FullOrthoDF, left_on='HomologName', right_on='SpeciesName')

In [ ]:
def count_peak_ADs(tempDF):
    # Prominence was 38000
    peaks, _ = find_peaks(tempDF["SmoothedActivitesLoess"], prominence=25000, height=ActivityThresh, distance=15)
    # print(peaks)
    if tempDF["SmoothedActivitesLoess"][0] > ActivityThresh + 10000:
        # Making sure we haven't already called this peak
        if len(peaks) == 0 or (tempDF["SmoothedActivitesLoess"][:peaks[0]] <  tempDF["SmoothedActivitesLoess"][0] - 7000).any():
            peaks = np.append(peaks, 0)

    return peaks, len(peaks)

In [ ]:
fig, axs = plt.subplots(1,1)

make_activity_trace("Anid_AN3675", "Plectosphaerella cucumerina", axs)

In [ ]:
# Try to make one big figure
rows = 63
columns = 8 
fig, axs = plt.subplots(rows, columns, figsize=[12*columns,6*rows])

i = 0
j = 0

for species in name_key["Species"].unique():
    tempSpeciesDF = name_key[name_key["Species"] == species].sort_values("Length", ascending=False)

    for name in tempSpeciesDF["HomologName"]:
        make_activity_trace(name, species, axs[i, j])
        if j + 1 == columns:
            j = 0
            i+=1
        else:
            j += 1

while (i < rows) & (j < columns):
    axs[i, j].remove()
    if j + 1 == columns:
        j = 0
        i+=1
    else:
        j += 1


plt.savefig(f'figures/all_traces_w_peak_count.pdf', format="pdf", bbox_inches='tight')
plt.savefig(f'figures/all_traces_w_peak_count.png', format="png", bbox_inches='tight')


In [ ]:
# Try to make one big figure
rows = 14
columns = 5 
fig, axs = plt.subplots(rows, columns, figsize=[12*columns,6*rows])

i = 0
j = 0
iteration = 1

for species in name_key["Species"].unique():
    tempSpeciesDF = name_key[name_key["Species"] == species].sort_values("Length", ascending=False)

    for name in tempSpeciesDF["HomologName"]:
        # print(i, j)
        make_activity_trace(name, species, axs[i, j])
        if j + 1 == columns:
            j = 0
            i+=1
        else:
            j += 1

        if (i + 1 > rows): 
            # print(i, j)
            plt.savefig(f'figures/all_traces_w_peak_count_v{iteration}.pdf', format="pdf", bbox_inches='tight')
            plt.savefig(f'figures/all_traces_w_peak_count_v{iteration}.png', format="png", bbox_inches='tight')
            iteration += 1
            i = 0
            j = 0
            fig, axs = plt.subplots(rows, columns, figsize=[12*columns,6*rows])

while (i < rows) & (j < columns):
    axs[i, j].remove()
    if j + 1 == columns:
        j = 0
        i+=1
    else:
        j += 1


plt.savefig(f'figures/all_traces_w_peak_count_v{iteration}.pdf', format="pdf", bbox_inches='tight')
plt.savefig(f'figures/all_traces_w_peak_count_v{iteration}.png', format="png", bbox_inches='tight')


In [ ]:
# How max calculated number of ADs 
HighActivityThresh = 150000
plothighactive = False 

tempDict ={}
# capture stats about TFs
NumActive,NumHighActive, lengths = [],[],[]
NumADs,NumHighADs =[],[]
NumTFswOneAD,NumTFswOneAD_high,NumTFswZeroAD_high =0,0,0
Collect_list_of_ADs =[]

FullTFSeqDict = dict(zip(FullOrthoDF.SpeciesName,FullOrthoDF.Seq))

#capture ADs that don't overlap the WxxLF motif
plt.figure(figsize=(10,2))
ADseqs, SpeciesNames = [],[]
TileLength = 40
genenames, starts,ends, seqs,regionlengths,regionlengthsHigh,RegionType = [],[],[],[],[],[],[]
SingleADoverlapsWxxLF,SingleADoverlapsWxxLF_Strong =0,0
LengthDict = dict(zip(FullOrthoDF.SpeciesName,FullOrthoDF.Length))

names = []
# Going through the tiles
for ii, name in enumerate( set(Orthologs_DF.SpeciesNames)):
    names.append(name)
    indx = Orthologs_DF.SpeciesNames==name
    
    tempDF = Orthologs_DF[indx] # DF for all tiles for each ortholog

    lengths.append(LengthDict[name])
    #combine overlapping tiles to estimate number of ADs. 
    tempDF2 = tempDF[tempDF.Activity>ActivityThresh]#active tiles
    tempDF2 = tempDF2.sort_values(by='NativeLocation',ascending=True)
    tempDF2 = tempDF2.reset_index()
    #check if there are no ADs
    if len(tempDF2)==0:
        NumADs.append(0)
        print(name+' has no active ADs')
    else:
        tempNumADs =1
        previousstart = (tempDF2.NativeLocation[0]-19)
        previousend   = (tempDF2.NativeLocation[0]+20)
        for i, entry in tempDF2.iterrows():
            if (entry.NativeLocation-19) < (previousend):
                previousend = entry.NativeLocation+20
            else:
                starts.append(previousstart)
                ends.append(previousend)
                tempNumADs +=1
                
                #restart
                previousstart = entry.NativeLocation-19
                previousend   = entry.NativeLocation+20
            # get seq of the AD 
            fullseq = FullTFSeqDict[name]

        NumADs.append(tempNumADs)      


In [ ]:
def get_overlapping_tile_ADs(tile_tempDF):
    ADs = []
    AD_end = 0
    AD_start = 0

    for i in tile_tempDF.index: 
        if tile_tempDF.loc[i, "Activity"] > ActivityThresh:
            # If start of new active tile is after the end of last active tile
            if tile_tempDF.loc[i, "NativeLocation"] - 19 > AD_end:
                # Save old AD and start a new AD
                if AD_end > 0: 
                    ADs.append((AD_start, AD_end))
                # Make new AD
                AD_end = tile_tempDF.loc[i, "NativeLocation"] + 20
                AD_start = tile_tempDF.loc[i, "NativeLocation"] - 19

            # Extend current AD
            else: 
                AD_end = tile_tempDF.loc[i, "NativeLocation"] + 20

    # Add the final AD to the list
    if AD_end > 0: 
        ADs.append((AD_start, AD_end))
    
    return ADs


In [ ]:
# Slight difference but I think we want to use > here because we want to be inclusive of the start? 
for name, AD_count in zip(names, NumADs):
    df = Orthologs_DF[Orthologs_DF["SpeciesNames"] == name].sort_values("NativeLocation").dropna(subset="Activity")
    my_count = len(get_overlapping_tile_ADs(df))
    if AD_count != my_count:
        print(name, AD_count, my_count)